# DSE Mini Project

First, data in csv files are read into DataFrame.

In [344]:
import pandas as pd
import matplotlib.pyplot as plt

# read data from csv
customers = pd.read_csv("data/customers_dataset.csv")
geolocation = pd.read_csv("data/geolocation_dataset.csv")
order_items = pd.read_csv("data/order_items_dataset.csv")
order_payments = pd.read_csv("data/order_payments_dataset.csv")
order_reviews = pd.read_csv("data/order_reviews_dataset.csv")
orders = pd.read_csv("data/orders_dataset.csv")
product_category = pd.read_csv("data/product_category_name_translation.csv")
products = pd.read_csv("data/products_dataset.csv")
sellers = pd.read_csv("data/sellers_dataset.csv")

Next, duplicates data in each DataFrame will be removed.

In [345]:
# drop duplicates
customers.drop_duplicates(inplace=True)
geolocation.drop_duplicates(inplace=True)
order_items.drop_duplicates(inplace=True)
order_payments.drop_duplicates(inplace=True)
order_reviews.drop_duplicates(inplace=True)
orders.drop_duplicates(inplace=True)
product_category.drop_duplicates(inplace=True)
products.drop_duplicates(inplace=True)
sellers.drop_duplicates(inplace=True)

In order to analyse data in depth, related DataFrames will be merged into one.

We will be mainly working on orders (orders and order_items), products (products and product_category), customers and sellers data.

In [346]:
# merge data
# main dataset
df = pd.merge(left=orders, right=order_items, on='order_id', how='inner')
df = df.merge(products, on='product_id', how='left')
df = df.merge(product_category, on='product_category_name', how='left')
df = df.merge(customers, on='customer_id', how='left')
df = df.merge(sellers, on='seller_id', how='left')

Geolocation data have different latitude and longitude on same zip code prefix, city and state which is confusing, therefore we will get the mean to represent the location data for particular zip code prefix, city and state.

In [347]:
# get the mean of latitude and longitude group by zip code prefix, city and state
geolocation_latlng_mean = geolocation.groupby(['geolocation_zip_code_prefix','geolocation_city','geolocation_state']).mean()
# convert the group by columns (index) to individual columns
geolocation_latlng_mean.reset_index(inplace=True)

In [348]:
#example data
geolocation_latlng_mean[(geolocation_latlng_mean['geolocation_zip_code_prefix'] == 1001) | (geolocation_latlng_mean['geolocation_zip_code_prefix'] == 79750)]

,geolocation_zip_code_prefix,geolocation_city,geolocation_state,geolocation_lat,geolocation_lng
0,1001,sao paulo,SP,-23.550312,-46.634030
1,1001,são paulo,SP,-23.549998,-46.634060
24099,79750,nova andradina,MS,-22.248022,-53.347396
24100,79750,nova andradina,RS,-22.242062,-53.343159


From above data we can still see that under one zip code prefix, there are different cities and states. Dropping either is not a good way as we don't know which data has the correct value. Instead, we will compare it against customers and sellers data to get the correct values.

In [349]:
# merge geolocation for customer and seller (join with zip code prefix, city and state)
df = df.merge(geolocation_latlng_mean, left_on=['customer_zip_code_prefix','customer_city','customer_state'], right_on=['geolocation_zip_code_prefix','geolocation_city','geolocation_state'], how='left')
df = df.merge(geolocation_latlng_mean, left_on=['seller_zip_code_prefix','seller_city','seller_state'], right_on=['geolocation_zip_code_prefix','geolocation_city','geolocation_state'], how='left')

# drop right joined columns (zip code prefix, state, city) since can viewed from left columns
df.drop(['geolocation_zip_code_prefix_x','geolocation_city_x','geolocation_state_x','geolocation_zip_code_prefix_y','geolocation_city_y','geolocation_state_y'], inplace=True, axis=1)

# rename columns suffix x to customer and y to seller for better understanding
df.rename(columns={'geolocation_lat_x': 'customer_geolocation_lat', 'geolocation_lng_x': 'customer_geolocation_lng', 'geolocation_lat_y': 'seller_geolocation_lat', 'geolocation_lng_y': 'seller_geolocation_lng'}, inplace=True)

An order can have multiple reviews and payments, therefore review and payment data are split to different dataset for easier analysis later.

In [350]:
# sub dataset
df_review = df.merge(order_reviews, on='order_id', how='left')
df_payment = df.merge(order_payments, on='order_id', how='left')